# 10 Minutes to cuDF and CuPy

This notebook provides introductory examples of how you can use cuDF and CuPy together to take advantage of CuPy array functionality (such as advanced linear algebra operations).

### Converting a cuDF DataFrame or Series to a CuPy Array

If we want to convert a cuDF `DataFrame` to a CuPy `ndarray`, the best way is to use the [DLPack interface](https://github.com/dmlc/dlpack).

In [1]:
import time

import numpy as np
import cupy as cp
import cudf
from numba import cuda

In [2]:
nelem = 10000
df = cudf.DataFrame({'a':range(nelem),
                     'b':range(500, nelem + 500),
                     'c':range(1000, nelem + 1000)}
                   )

%time arr_cupy = cp.fromDlpack(df.to_dlpack())
type(arr_cupy)

CPU times: user 1.55 ms, sys: 2.29 ms, total: 3.84 ms
Wall time: 103 ms


/home/umashankar/.conda/envs/python3/lib/python3.7/site-packages/cudf/io/dlpack.py:82: UserWarning: WARNING: cuDF to_dlpack() produces column-major (Fortran order) output. If the output tensor needs to be row major, transpose the output of this function.
  return cpp_dlpack.to_dlpack(gdf_cols)


cupy.core.core.ndarray

The best way to convert a cuDF `Series` to a CuPy `ndarray` is to either pass the underlying Numba `DeviceNDArray` to `cupy.asarray` to leverage the [CUDA Array Interface](https://numba.pydata.org/numba-doc/dev/cuda/cuda_array_interface.html) or leverage the `dlpack` interface for conversions. We can also pass the `Series` itself, but this will be far slower. We're working on adding the `__cuda_array_interface__` attribute to `Series`, so eventually you'll be able to pass the Series directly with low latency (see [this issue](https://github.com/rapidsai/cudf/issues/2433) to track our progress).

In [3]:
col = 'a'

%time cola_cupy = cp.asarray(df[col].data.mem)
%time cola_cupy = cp.fromDlpack(df[col].to_dlpack())
%time cola_cupy = cp.asarray(df[col])
type(cola_cupy)

CPU times: user 2.35 ms, sys: 0 ns, total: 2.35 ms
Wall time: 42 ms
CPU times: user 2.19 ms, sys: 0 ns, total: 2.19 ms
Wall time: 1.27 ms
CPU times: user 315 µs, sys: 0 ns, total: 315 µs
Wall time: 321 µs


cupy.core.core.ndarray

From here, we can proceed with normal CuPy workflows, such as reshaping the array, getting the diagonal, or calculating the norm.

In [4]:
reshaped_arr = cola_cupy.reshape(50, 200)
reshaped_arr

array([[   0,    1,    2, ...,  197,  198,  199],
       [ 200,  201,  202, ...,  397,  398,  399],
       [ 400,  401,  402, ...,  597,  598,  599],
       ...,
       [9400, 9401, 9402, ..., 9597, 9598, 9599],
       [9600, 9601, 9602, ..., 9797, 9798, 9799],
       [9800, 9801, 9802, ..., 9997, 9998, 9999]])

In [5]:
reshaped_arr.diagonal()

array([   0,  201,  402,  603,  804, 1005, 1206, 1407, 1608, 1809, 2010,
       2211, 2412, 2613, 2814, 3015, 3216, 3417, 3618, 3819, 4020, 4221,
       4422, 4623, 4824, 5025, 5226, 5427, 5628, 5829, 6030, 6231, 6432,
       6633, 6834, 7035, 7236, 7437, 7638, 7839, 8040, 8241, 8442, 8643,
       8844, 9045, 9246, 9447, 9648, 9849])

In [6]:
cp.linalg.norm(reshaped_arr)

array(577306.967739)

### Converting a CuPy Array to a cuDF DataFrame or Series

We can also convert a CuPy `ndarray` to a cuDF `DataFrame` or `Series`. We can use the same `dlpack` interface from above, or rely on the `__cuda_array_interface__` and use cuDF's `from_gpu_matrix`. Either way, we'll need to make sure that our CuPy array is Fortran contiguous in memory (if it's not already). We can either transpose the array or simply coerce it to be Fortran contiguous beforehand.

We can check whether our array is Fortran contiguous by using `cp.isfortran` or looking at the [flags](https://docs-cupy.chainer.org/en/stable/reference/generated/cupy.ndarray.html#cupy.ndarray.flags) of the array.

In [7]:
cp.isfortran(reshaped_arr)

False

In this case, we'll need to convert it before going to cuDF.

In [8]:
reshaped_arr = cp.asfortranarray(reshaped_arr)
reshaped_df = cudf.from_dlpack(reshaped_arr.toDlpack())
print(reshaped_df.head())

/home/umashankar/.conda/envs/python3/lib/python3.7/site-packages/cudf/io/dlpack.py:35: UserWarning: WARNING: cuDF from_dlpack() assumes column-major (Fortran order) input. If the input tensor is row-major, transpose it before passing it to this function.
  res, valids = cpp_dlpack.from_dlpack(pycapsule_obj)


   0    1    2    3    4    5    6    7    8    9    ...  190  191  192  193  \
0    0    1    2    3    4    5    6    7    8    9  ...  190  191  192  193   
1  200  201  202  203  204  205  206  207  208  209  ...  390  391  392  393   
2  400  401  402  403  404  405  406  407  408  409  ...  590  591  592  593   
3  600  601  602  603  604  605  606  607  608  609  ...  790  791  792  793   
4  800  801  802  803  804  805  806  807  808  809  ...  990  991  992  993   

   194  195  196  197  198  199  
0  194  195  196  197  198  199  
1  394  395  396  397  398  399  
2  594  595  596  597  598  599  
3  794  795  796  797  798  799  
4  994  995  996  997  998  999  

[5 rows x 200 columns]


In [9]:
reshaped_df = cudf.DataFrame.from_gpu_matrix(reshaped_arr)
print(reshaped_df.head())

   0    1    2    3    4    5    6    7    8    9    ...  190  191  192  193  \
0    0    1    2    3    4    5    6    7    8    9  ...  190  191  192  193   
1  200  201  202  203  204  205  206  207  208  209  ...  390  391  392  393   
2  400  401  402  403  404  405  406  407  408  409  ...  590  591  592  593   
3  600  601  602  603  604  605  606  607  608  609  ...  790  791  792  793   
4  800  801  802  803  804  805  806  807  808  809  ...  990  991  992  993   

   194  195  196  197  198  199  
0  194  195  196  197  198  199  
1  394  395  396  397  398  399  
2  594  595  596  597  598  599  
3  794  795  796  797  798  799  
4  994  995  996  997  998  999  

[5 rows x 200 columns]


If the array is not already contiguous, we'll need to create a contiguous array with `ascontiguousarray`.

In [10]:
diag_data = cp.ascontiguousarray(reshaped_arr.diagonal())
print(cudf.Series(diag_data))

0        0
1      201
2      402
3      603
4      804
5     1005
6     1206
7     1407
8     1608
9     1809
10    2010
11    2211
12    2412
13    2613
14    2814
15    3015
16    3216
17    3417
18    3618
19    3819
20    4020
21    4221
22    4422
23    4623
24    4824
25    5025
26    5226
27    5427
28    5628
29    5829
30    6030
31    6231
32    6432
33    6633
34    6834
35    7035
36    7236
37    7437
38    7638
39    7839
40    8040
41    8241
42    8442
43    8643
44    8844
45    9045
46    9246
47    9447
48    9648
49    9849
dtype: int64


### Interweaving CuDF and CuPy for Smooth PyData Workflows

RAPIDS libraries and the entire GPU PyData ecosystem are developing quickly, but sometimes a one library may not have the functionality you need. One example of this might be taking the row-wise sum (or mean) of a Pandas DataFrame. cuDF's support for row-wise operations isn't mature, so you'd need to either transpose the DataFrame or write a UDF and explicitly calculate the sum across each row. Transposing could lead to hundreds of thousands of columns (which cuDF wouldn't perform well with) depending on your data's shape, and writing a UDF can be time intensive.

By leveraging the interoperability of the GPU PyData ecosystem, this operation becomes very easy. Let's take the row-wise sum of our previously reshaped cuDF DataFrame.

In [11]:
print(reshaped_df.head())

   0    1    2    3    4    5    6    7    8    9    ...  190  191  192  193  \
0    0    1    2    3    4    5    6    7    8    9  ...  190  191  192  193   
1  200  201  202  203  204  205  206  207  208  209  ...  390  391  392  393   
2  400  401  402  403  404  405  406  407  408  409  ...  590  591  592  593   
3  600  601  602  603  604  605  606  607  608  609  ...  790  791  792  793   
4  800  801  802  803  804  805  806  807  808  809  ...  990  991  992  993   

   194  195  196  197  198  199  
0  194  195  196  197  198  199  
1  394  395  396  397  398  399  
2  594  595  596  597  598  599  
3  794  795  796  797  798  799  
4  994  995  996  997  998  999  

[5 rows x 200 columns]


We can just transform it to a CuPy array via `dlpack` and use the `axis` argument of `sum`.

In [12]:
new_arr = cp.fromDlpack(reshaped_df.to_dlpack())
new_arr.sum(axis=1)

array([  19900,   59900,   99900,  139900,  179900,  219900,  259900,
        299900,  339900,  379900,  419900,  459900,  499900,  539900,
        579900,  619900,  659900,  699900,  739900,  779900,  819900,
        859900,  899900,  939900,  979900, 1019900, 1059900, 1099900,
       1139900, 1179900, 1219900, 1259900, 1299900, 1339900, 1379900,
       1419900, 1459900, 1499900, 1539900, 1579900, 1619900, 1659900,
       1699900, 1739900, 1779900, 1819900, 1859900, 1899900, 1939900,
       1979900])

With just that single line, we're able to seamlessly move between data structures in this ecosystem, giving us enormous flexibility without sacrificing speed.

### Converting a cuDF DataFrame to a CuPy Sparse Matrix.

We can also convert a `DataFrame` or `Series` to a CuPy sparse matrix. We might want to do this if downstream processes expect CuPy sparse matrices as inputs.

The sparse matrix data structure is defined by three dense arrays, which we could create manually from an existing cuDF `DataFrame` or `Series`. Luckily, we don't need to do that. We can simply leverage `dlpack` again. We'll define a small helper function for cleanliness.

In [13]:
def cudf_to_cupy_sparse_matrix(data, sparseformat='column'):
    """Converts a cuDF object to a CuPy Sparse Column matrix.
    """
    if sparseformat not in ('row', 'column',):
        raise ValueError("Let's focus on column and row formats for now.")
    
    _sparse_constructor = cp.sparse.csc_matrix
    if sparseformat == 'row':
        _sparse_constructor = cp.sparse.csr_matrix

    return _sparse_constructor(cp.fromDlpack(data.to_dlpack()))

We can define a sparsely populated dataframe to illustrate this conversion to either sparse matrix format.

In [14]:
df = cudf.DataFrame()
nelem = 10000
nonzero = 1000
for i in range(20):
    arr = cp.random.normal(5, 5, nelem)
    arr[cp.random.choice(arr.shape[0], nelem-nonzero, replace=False)] = 0
    df['a' + str(i)] = arr

In [15]:
print(df.head())

              a0        a1             a2        a3             a4        a5  \
0  4.980182e-321  0.000000  4.980182e-321  0.000000  4.980182e-321  0.000000   
1  3.191170e-320  0.000000  3.191170e-320  0.000000  3.191170e-320  0.000000   
2  2.413017e-320  0.000000  2.413017e-320  0.000000  2.413017e-320  0.000000   
3  8.769665e-321  4.990832  8.769665e-321  4.990832  8.769665e-321  4.990832   
4  2.555802e-320  7.836822  2.555802e-320  7.836822  2.555802e-320  7.836822   

              a6        a7             a8        a9            a10       a11  \
0  4.980182e-321  0.000000  4.980182e-321  0.000000  4.980182e-321  0.000000   
1  3.191170e-320  0.000000  3.191170e-320  0.000000  3.191170e-320  0.000000   
2  2.413017e-320  0.000000  2.413017e-320  0.000000  2.413017e-320  0.000000   
3  8.769665e-321  4.990832  8.769665e-321  4.990832  8.769665e-321  4.990832   
4  2.555802e-320  7.836822  2.555802e-320  7.836822  2.555802e-320  7.836822   

             a12       a13            

In [16]:
sparse_data = cudf_to_cupy_sparse_matrix(df)

From here, we could continue our workflow with a CuPy sparse matrix.

For a full list of the functionality built into these libraries, we encourage you to check out the API docs for [cuDF](https://docs.rapids.ai/api/cudf/nightly/) and [CuPy](https://docs-cupy.chainer.org/en/stable/index.html).